### 从es中读取数据，将数据写进文件，再调用百度接口进行分词。

## es连接

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import re
import sys
import time
from datetime import datetime
import random

es = Elasticsearch('http://ip:port')

## query设置-----全量读取

In [2]:
query = {
  "query": {
    "match_all": {}
  },
  "_source": {
    "includes": [
      "url",
      "title",
      "content"
    ]
  }
}

In [3]:
# 读取全部平台的数据（包括36kr(36kr_bl),百度(baidu_bl)，微信公众号(wechat_bl)，今日头条(toutiao_bl)）
doc_index_total = '*_bl'
doc_type_total = 'news'
dataGen_total = helpers.scan(es, 
                   query=query, 
                   index=doc_index_total, 
                   doc_type=doc_type_total, 
                   scroll="10m", 
                   raise_on_error=True, 
                   preserve_order=True,
                   timeout='10m')
data_total = list(dataGen_total)
# len(data_total) # 56026

In [6]:
# 读取36kr数据
doc_index = '36kr_bl'
doc_type = 'news'
dataGen = helpers.scan(es, 
                   query=query, 
                   index=doc_index, 
                   doc_type=doc_type, 
                   scroll="10m", 
                   raise_on_error=True, 
                   preserve_order=True,
                   timeout='10m')
data = list(dataGen)
# len(data) # 918

In [99]:
# data[0]['_source']['content']

## 解析html，返回句子

In [12]:
# 拆分为句子
def sentencesMaker(html):

    import justext
    paragraphs = justext.justext(html, [])

    cache_sentences = ''
    sentences = []

    for p in paragraphs:
        sent = p.text.strip().replace('\xa0', '').replace('\u3000', '')
        sent = sent.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk')
        if not sent:
            continue

        # 可能是含有名字，需要进一步处理
        if len(cache_sentences) < 5:
            cache_sentences += ' ' + sent

        else:
            sentences.append(cache_sentences.strip())
            cache_sentences = sent

    if not not cache_sentences:
        sentences.append(cache_sentences.strip())

    return sentences

In [16]:
# print(type(data_total[0]))

In [97]:
# print("\n".join((sentencesMaker(data[0]['_source']['content']))))

## 从es中读取的所有数据写入到一个txt中

In [82]:
# 写36kr数据，剔除content=null的数据

dataset = open('data_36kr.txt', 'w', encoding='utf-8')
i = 1
for item in data:
    source = item['_source']
    title = source['title']
    url = source['url']
    content = "".join(sentencesMaker(source['content']))
    if content:
        dataset.write('sentence%s:' % str(i) + '\r\n')
        dataset.write('url:'+' '+url+'\r\n')
        dataset.write('title:'+' '+title+'\r\n')
        dataset.write('content:'+' '+str(content)+'\r\n')
        dataset.write('\n')
        i += 1
    else:
        dataset.write('\n')
dataset.close()

In [ ]:
# 写所有数据，剔除content=null的数据
# 共55397行，写入了一个.txt文件中，dataTotal.txt
# 格式为 sentence1：
#       url:
#       title:
#       content:
# 不去除标识(url\title\content)

dataset1 = open('dataTotal.txt', 'w', encoding='utf-8')
i = 1    
for item in data_total:
    try:
        source = item['_source']
        title = source.get('title',None) # 返回指定键的值，如果值不在字典中返回默认值None。
        html = source.get('content', None)
        if html:
            html = html.strip()
        if not html or not title: # 如果html或title有一个为空，跳出本次循环
            continue
        if not isinstance(html, str): # 判断html是否是str类型，如果不是，跳出本次循环
            continue
        url = source['url']
        content = "".join(sentencesMaker(html)).strip()
        if content:
            dataset1.write('sentence%s:' % str(i) + '\r\n')
            dataset1.write('url:'+' '+url+'\r\n')
            dataset1.write('title:'+' '+title+'\r\n')
            dataset1.write('content:'+' '+str(content)+'\r\n')
            dataset1.write('\n')
            i += 1
        else:
            dataset1.write('\n')
    except Exception as e:
        print(item)
        raise e
dataset1.close()

In [112]:
# 写所有数据，剔除content=null的数据
# 共55397行，写入了一个.txt文件中，dataTotal2.txt
# 格式为 url:
#       title+content:
# 去掉标识(url、title+content)

dataset2 = open('dataTotal2.txt', 'w', encoding='utf-8')
# i = 1    
for item in data_total:
    try:
        source = item['_source']
        title = source.get('title',None) # 返回指定键的值，如果值不在字典中返回默认值None。
        html = source.get('content', None)
        if html:
            html = html.strip()
        if not html or not title:
            continue
        if not isinstance(html, str): # 判断html是否是str类型
            continue
        url = source['url']
        content = "".join(sentencesMaker(html)).strip()
        if content:
#             dataset1.write('sentence%s:' % str(i) + '\r\n')
            dataset2.write(url+'\r\n')
            dataset2.write(title+'。'+content+'\r\n')
#             dataset2.write('content:'+' '+str(content)+'\r\n')
            dataset2.write('\n')
#             i += 1
        else:
            dataset2.write('\n')
    except Exception as e:
        print(item)
        raise e
dataset2.close()

## 将获取的每篇数据写入一个.txt文件中

In [113]:
# 按每三行写一篇.txt，后来发现数据格式有点比较差，不能这么写。

# index = 0
# count = 0
# f_in = open("./data/%d.txt" % index, "w")
# with open("dataTotal2.txt", "r") as f_out:
#     for line in f_out:
#         count += 1
#         f_in.write(line)

#         # 读满3行之后，行计数置零，小文件序号加1，创建一个新的文件写信息
#         if count == 3:
#             f_in.close()
#             count = 0
#             index += 1
#             f_in = open("./data/%d.txt" % index, "w")


In [9]:
# 按遇到下一个sentence时，停止写文件，较准确

count = 0
index = 0
f_in = open("./data/%d.txt" % index, "w")
with open("dataTotal.txt", "r") as f_out:
    for line in f_out:
        line = line.strip()
        count += 1       
        if count > 1 and line.startswith("sentence"):
            f_in.close()
            count = 0
            index += 1
            f_in = open("./data/%d.txt" % index, "w")
        else:
            f_in.write(line+'\r\n')          

In [32]:
# 查看文件目录下共写了多少个txt，查看每个txt的命名是否有误
import os
print(len(os.listdir("./data")))
# 文件目录下多了个.ipynb_checkpoints，要删除，否则会报错
# os.rmdir("./data/.ipynb_checkpoints")
print(os.listdir("./data")[:20])

55399
['0.txt', '1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt', '10.txt', '11.txt', '12.txt', '13.txt', '14.txt', '15.txt', '16.txt', '17.txt', '18.txt', '19.txt']


## 百度api，句子拆分与合并

In [4]:
from aip import AipNlp

APP_ID = '11513666'
API_KEY = 'Tpw3HSxiWqPZbm5tqRRQDd6H'
SECRET_KEY = 'tOuMo5GNXEh6GyDVokSAkjMWVZFKAjHm'

client = AipNlp(APP_ID, API_KEY, SECRET_KEY)

In [5]:
# 改善QPS限制

import time
import sys
import re
SPLIT_LINE_MARKER = '！'
SPLIT_LINE_MARKER_SIZE = 3

# 拆分为句子
# def sentencesMaker(html):

#     import justext
#     paragraphs = justext.justext(html, [])

#     cache_sentences = ''
#     sentences = []


#     for p in paragraphs:
#         sent = p.text.strip().replace('\xa0', '').replace('\u3000', '')
#         sent = sent.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk')
#         if not sent:
#             continue

#         # 可能是含有名字，需要进一步处理
#         if len(cache_sentences) < 5:
#             cache_sentences += ' ' + sent

#         else:
#             sentences.append(cache_sentences.strip())
#             cache_sentences = sent

#     if not not cache_sentences:
#         sentences.append(cache_sentences.strip())

#     return sentences

# 重新恢复句子
def restoreSentences(text, only_per=False):  # 长度不超过3700字节的句子
    restore_sentences = []
    isSucc = False
    if text is None:
        return restore_sentences, isSucc

    result = client.lexerCustom(text)

    items = result.get('items', [])
    items_size = len(items)

    tries_limit = 3
    tries_counter = 0
    
    while items_size == 0: # 分词结果为空
        if len(text) != 0: # 但句子长度不为空
            # 可能是qps限制
            time.sleep(1)
            result = client.lexerCustom(text)

            items = result.get('items', [])
            items_size = len(items)
            isSucc=True

        tries_counter += 1

        if tries_counter >= tries_limit: # 分词尝试大于等于4次之后仍失败
            print(f'error: 分词api请求失败多次！{text}')
#             print('error: 分词api请求失败多次!')
            return restore_sentences, isSucc

    restore_idx = 0

    last_restore_idx = 0
    has_per = False

    while restore_idx < items_size: # 对每个分词结果进行整理 
        # 分词不是拼接符"!!!"
        while restore_idx < items_size and items[restore_idx]['item'] != SPLIT_LINE_MARKER:
            item = items[restore_idx] # 先把第一个分词的结果(dict)赋值给item，item整理之后再直接赋给原items[]
            # TODO 剔除机构中的不合法字符
            format_pos = item['pos']
            
            # 对非ne标识的分词不做处理
            
            if item['ne'].startswith('ORG'): # 如果该分词的ne是ORG
                invalid_orgs = ['公司']
                item['item'] = item['item'].replace('&', '')
                if item['item'] in invalid_orgs:
                    format_pos = 'n' # 普通名词
                else:
                    format_pos = 'nt'  # 机构团体名

            elif item['ne'] == 'PER': # 如果该分词的ne是PER
                format_pos = 'nr' # 人名

            elif item['ne'] == 'TITLE': # 如果该分词的ne是TITLE（定制）
                format_pos = 'ti' # 职称

            elif item['ne'] == 'LOC': # 如果该分词的ne是LOC
                format_pos = 'ns'  # 地名

            elif item['ne'] == 'TIME': # 如果该分词的ne是TIME
                format_pos = 't' # 时间名词
                
                

            if format_pos == '': #如果pos为空，即是其他非上述的ne标识，将pos置为"xx"
                format_pos = 'xx'

            elif format_pos == 'nr':
                # 过滤先生或者女士之类的名称
                name = re.sub(r'((先生)|(小姐)|(阿姨)|(叔叔)|(女士)|(同志)|总)$', '', item['item'])

                if len(name) >= 2:
                    invalid_names = {
                        '区块链': 'n'
                    }

                    if name not in invalid_names:
                        has_per = True
                        item['item'] = name
                    else:
                        format_pos = invalid_names[name]

                else: # 剔除称谓之后的name长度如果小于2，就不是nr，设为n

                     format_pos = 'n'




            item['pos'] = format_pos # 处理之后的pos赋值给原分词的pos

            # 删除无用信息
            item.pop('basic_words')
            item.pop('formal')
            item.pop('byte_length')
            item.pop('byte_offset')
            item.pop('loc_details')
            item.pop('ne')
            item.pop('uri')

            items[restore_idx] = item # 处理之后的分词结果赋给原分词结果
            restore_idx += 1 # 继续下一个item

            
        # 若遇到了拼接符"!!!"或对所有非拼接符的分词结果处理完毕    
        if restore_idx + SPLIT_LINE_MARKER_SIZE - 1 < items_size: 
            needCut = True 
            for i in range(SPLIT_LINE_MARKER_SIZE - 1): 
                if items[restore_idx + i + 1]['item'] != SPLIT_LINE_MARKER:
                    needCut = False
                    break

            if needCut:
                ed = max(restore_idx, 0)

                sentence_items = items[last_restore_idx:ed] # 切句子，[0:restore_idx]
                if len(sentence_items) != 0 and (has_per or not only_per):
                    # print('per:', sentence_items)
                    restore_sentences.append(sentence_items)

                next_st = min(ed + SPLIT_LINE_MARKER_SIZE, items_size)
                last_restore_idx = next_st

                restore_idx += SPLIT_LINE_MARKER_SIZE
            else:
                restore_idx += 1

        else: # 句末最后一个拼接符
            ed = max(restore_idx, 0)
            sentence_items = items[last_restore_idx:ed]
            if len(sentence_items) != 0 and (has_per or not only_per):

                restore_sentences.append(sentence_items)

            restore_idx = items_size

        has_per = False

    # print(restore_sentences)
    return restore_sentences, isSucc


# 解析并标注HTML
def posHtml(sentences, only_per=False):
#     sentences = sentencesMaker(html) # 将带有html标签的段落整理成一个list,去除标签的文字段落。


    cut_str = '' 

    pos_sentences = []

    for sent in sentences:
        sent = sent.strip()
        if not sent:  # 若句子为空，跳出循环
            continue
        if not cut_str: # 若cut_str为空，即第一句之前，把第一句话赋值给tmp_str
            tmp_str = sent
        else: # 若cut_str里有句子，将后续句子拼接，以“!!!”作为拼接符，再赋值给tmp_str
            tmp_str = cut_str + SPLIT_LINE_MARKER * SPLIT_LINE_MARKER_SIZE + sent
        if sys.getsizeof(tmp_str) < 3700: # 拼接之后的句子小于3700字节
            cut_str = tmp_str # tmp_str赋值给cut_str，继续拼接sent
        else:
            try:
                if cut_str:
                    time.sleep(0.5) # 拼接之后的tmp_str若大于3700字节，取未拼接该sent之前的cut_str拿来分词
                    sents, issucc = restoreSentences(cut_str, only_per)
                    pos_sentences += sents
    #                 if not issucc:
    #                     print(html)

            except Exception as e:
                print(e)
                print('error: ', sent)

            cut_str = sent # 把刚才没有拼接成功的sent重新赋给cut_str


    if not not cut_str: # 最后一句sent
        time.sleep(0.5)
        try:
            if cut_str:
                time.sleep(0.5) # 拼接之后的tmp_str若大于3700字节，取未拼接该sent之前的cut_str拿来分词
                sents, issucc = restoreSentences(cut_str, only_per)
                pos_sentences += sents
#                 if not issucc:
#                     print(html)
        except Exception as e:
            print(e)
            print('error: ', sent)

    return pos_sentences

In [6]:
restoreSentences("这是一个测试的小句子")

([[{'item': '这', 'pos': 'r'},
   {'item': '是', 'pos': 'v'},
   {'item': '一个', 'pos': 'm'},
   {'item': '测试', 'pos': 'v'},
   {'item': '的', 'pos': 'u'},
   {'item': '小', 'pos': 'a'},
   {'item': '句子', 'pos': 'n'}]],
 False)

In [7]:
posHtml("这是一个测试的小句子")

[[{'item': '这', 'pos': 'r'}],
 [{'item': '是', 'pos': 'v'}],
 [{'item': '一', 'pos': 'm'}],
 [{'item': '个', 'pos': 'q'}],
 [{'item': '测', 'pos': 'v'}],
 [{'item': '试', 'pos': 'v'}],
 [{'item': '的', 'pos': 'u'}],
 [{'item': '小', 'pos': 'a'}],
 [{'item': '句', 'pos': 'n'}],
 [{'item': '子', 'pos': 'n'}]]

## 将./data数据7/3份划分训练集和测试集

In [30]:
# import os,shutil
            
# i = 0            
# ls = os.listdir("./data")
# try:
#     for line in ls:
#         filePath = os.path.join("./data", line)
#         if os.path.isfile(filePath):
#             if i < 38778:
#                 shutil.copy(filePath, "./trainData")
#                 i += 1
#             else:
#                 shutil.copy(filePath, "./testData")
#         else:
#             continue
# except Exception as e:
#     print(e)

In [42]:
list_c = [a for a in trainData if a in testData]
print(list_c)  # 训练集和测试集没有重复的文件

[]


## 批量读取训练集的分词结果

In [133]:
import os
import json
# import shutil
# shutil.rmtree("./data/.ipynb_checkpoints")
total_number = 0
# wrong_number = 0
log = open('log1.txt', 'w', encoding='utf-8')
f = open('corpus2.json', 'w', encoding="utf-8")
i = 0

caches = []
for file in os.listdir(r'./data'):
    domain = os.path.abspath(r'./data')
    file = os.path.join(domain, file) # 获取trainData下的所有文件
#     print(file)
    if i < 5:
        data = open(file, "r", encoding="utf-8")
        title = ''
        content = ''
        para = []
        for line in data:
            line = line.strip()
            if line.startswith("title"):
                title = line[6:]
            if line.startswith("content"):
                content = line[8:]
        text = title + content
        text = text.split("。")
        for line in text:
            para.append(line)
        f.write(json.dumps(posHtml(para),ensure_ascii=False,indent=2))
        print(i, '******************************************')
        i += 1
        caches.append(text)
        total_number += 1
    else:
        continue
print("total_number", total_number)

0 ******************************************
1 ******************************************
2 ******************************************
3 ******************************************
4 ******************************************
total_number 5


In [109]:
client.lexerCustom(caches[3])

{'log_id': 8378757205452365828,
 'error_code': 282002,
 'error_msg': 'input encoding error'}

 ## 将分词结果转成word pos形式，不剔除重复项。

In [135]:
item = []
pos = []
flag = 0
corpus = open("./corpus2.txt", "w", encoding="utf-8")
with open("./corpus2.json", "r") as json_result:
    for line in json_result:
        line = line.strip()
        if "[" in line:
            continue
        if "{" in line:
            continue
        if "}" in line:
            continue
        if "]" in line:
            continue
        if "item" in line:
            item.append(line.replace(",","").split(":")[1].split('"')[1])
        elif "pos" in line and flag ==0:
            pos.append(line.split(":")[1].split('"')[1])
    dic = zip(item, pos)
    for key, value in dic:
        corpus.write(key+ " "+value + '\n')
    corpus.close()

## 将trainData分10次进行分词，转形式处理。
### es.py进行第1，3次处理；
#### 其中第1次处理的结果，即报gbk编码问题保存在本地 D:\Google Download\编码错误的部分。
#### 第2，3，...，10次处理的结果保存在控制台，暂未做处理。
#### trainData中之所以会出现那么多带！！！没有分词的段落是因为当时让9份分词程序同时跑，同时访问百度的API接口，所以会出现这个现象，后面再测试的时候，就几乎没有这个问题了，所以代码中split("！！！")这一步其实没有作用。
### train1.py进行第2次处理；
### train3.py进行第4次处理；
### .....................
### train9.py进行第10次处理；

In [ ]:
import os
import json
total_number = 0
f = open('trainData_111111.json', 'w', encoding="utf-8")
for file in os.listdir(r'./trainData'):
    number = int(file.split('.')[0]) 
    domain = os.path.abspath(r'./trainData')
    file = os.path.join(domain, file) 
    if 0 <= number <= 3878: 
        print(file)
        data = open(file, "r", encoding="utf-8")
        title = ''
        content = ''
        para = []
        for line in data:
            line = line.strip()
            try:
                if line.startswith("title"):
                    title = line[6:]
                if line.startswith("content"):
                    content = line[8:]
            except Exception as e:
                print(1)
                print(e)
        text = title + content
        text = text.split("。")
        for line in text:
            para.append(line)
        f.write(json.dumps(posHtml(para),ensure_ascii=False,indent=2))
#         caches.append(text)
        total_number += 1
    else:
        continue
print("total_number", total_number)

/home/lx/Projects/wjc/es/trainData/0.txt
/home/lx/Projects/wjc/es/trainData/1.txt
/home/lx/Projects/wjc/es/trainData/2.txt
/home/lx/Projects/wjc/es/trainData/3.txt
/home/lx/Projects/wjc/es/trainData/4.txt
/home/lx/Projects/wjc/es/trainData/5.txt
/home/lx/Projects/wjc/es/trainData/6.txt
/home/lx/Projects/wjc/es/trainData/7.txt
/home/lx/Projects/wjc/es/trainData/8.txt
/home/lx/Projects/wjc/es/trainData/9.txt
/home/lx/Projects/wjc/es/trainData/10.txt
/home/lx/Projects/wjc/es/trainData/11.txt
/home/lx/Projects/wjc/es/trainData/12.txt
/home/lx/Projects/wjc/es/trainData/13.txt
/home/lx/Projects/wjc/es/trainData/14.txt
/home/lx/Projects/wjc/es/trainData/15.txt
'gbk' codec can't encode character '\u2022' in position 57: illegal multibyte sequence
error:  我们希望这是我们的一个优势
/home/lx/Projects/wjc/es/trainData/16.txt
/home/lx/Projects/wjc/es/trainData/17.txt
/home/lx/Projects/wjc/es/trainData/18.txt
/home/lx/Projects/wjc/es/trainData/19.txt
/home/lx/Projects/wjc/es/trainData/20.txt
/home/lx/Projects/w

In [6]:
item = []
pos = []
flag = 0
corpus = open("./trainData_111111.txt", "w", encoding="utf-8")

with open("./trainData_111111.json", "r") as json_result:
    for line in json_result:
        line = line.strip()
        if "[" in line:
            continue
        if "{" in line:
            continue
        if "}" in line:
            continue
        if "]" in line:
            continue
        if "item" in line:
            item.append(line.replace(",","").split(":")[1].split('"')[1])
        elif "pos" in line and flag ==0:
            pos.append(line.split(":")[1].split('"')[1])
    dic = zip(item, pos)
    for key, value in dic:
        corpus.write(key+ " "+value + '\n')
    corpus.close()

## 将测试集2/1分为验证集和测试集

### 写反了 testData是38779-49858 valData是49859-55397

In [5]:
import shutil
for file in os.listdir(r'./testData'):
    number = int(file.split('.')[0]) 
    domain = os.path.abspath(r'./testData')  # 49859.txt-55397.txt
    file = os.path.join(domain, file)
    if 38779 <= number <= 49858: 
        shutil.copy(file, "./valData")  # 38779.txt-49858.txt
    else:
        continue

In [13]:
import shutil 
for file in os.listdir(r'./testData'):
    number = int(file.split('.')[0]) #获取文件名 *.txt 中的数字编号
    domain = os.path.abspath(r'./testData')
    file = os.path.join(domain, file) # 获取trainData下的所有文件
    if 38779 <= number <= 49858: 
        os.remove(file)

In [3]:
import json
json.loads(open('trainData_1.json').read())

[[{'item': 'AI', 'pos': 'nz'},
  {'item': '产品经理', 'pos': 'ti'},
  {'item': '需要', 'pos': 'v'},
  {'item': '了解', 'pos': 'vn'},
  {'item': '的', 'pos': 'u'},
  {'item': '语音', 'pos': 'n'},
  {'item': '交互', 'pos': 'vn'},
  {'item': '评价', 'pos': 'vn'},
  {'item': '指标', 'pos': 'n'},
  {'item': ' ', 'pos': 'w'},
  {'item': '编者按', 'pos': 'n'},
  {'item': '：', 'pos': 'w'},
  {'item': '本文', 'pos': 'n'},
  {'item': '来自', 'pos': 'v'},
  {'item': '微信公众号', 'pos': 'nz'},
  {'item': '"', 'pos': 'w'},
  {'item': 'hanniman', 'pos': 'nz'},
  {'item': '"', 'pos': 'w'},
  {'item': '（', 'pos': 'w'},
  {'item': 'ID', 'pos': 'nz'},
  {'item': '：', 'pos': 'w'},
  {'item': 'hanniman', 'pos': 'v'},
  {'item': '）', 'pos': 'w'},
  {'item': '，', 'pos': 'w'},
  {'item': '作者', 'pos': 'n'},
  {'item': '：', 'pos': 'w'},
  {'item': '黄钊', 'pos': 'nr'},
  {'item': 'hanniman', 'pos': 'v'},
  {'item': '，', 'pos': 'w'},
  {'item': '图灵机器人', 'pos': 'nz'},
  {'item': '-', 'pos': 'w'},
  {'item': '人才', 'pos': 'n'},
  {'item': '战略'